In [1]:
import pandas as pd
homeValueData = pd.read_csv('/content/drive/MyDrive/ColabData/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv')
rentalData = pd.read_csv('/content/drive/MyDrive/ColabData/Zip_zori_sm_month.csv')
propertyTax = pd.read_csv('/content/drive/MyDrive/ColabData/property-taxes-by-state-[updated-march-2023].csv')
insuranceCostByState = pd.read_csv('/content/drive/MyDrive/ColabData/homeowners_insurance_cost.csv')

In [2]:
#Refine Property Tax Data

#Add Missing Property Tax Value for Oregon
new_state = 'OR'
new_tax_rate = 0.082
propertyTax.at[len(propertyTax), 'state'] = new_state
propertyTax.at[len(propertyTax)-1, 'propertyTaxRate'] = new_tax_rate

#Replacing state name with abbreviation
propertyTax['state'] = propertyTax['state'].replace({'Ohio': 'OH'})
propertyTax.rename(columns={'state': 'State'}, inplace=True)

In [3]:
#Set Pandas Display Options
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

#Create Dataframe with the Home Values for May
MayHomeValues = homeValueData[['SizeRank', 'RegionName', 'State', 'City', 'Metro', 'CountyName', '2012-05-31', '2018-05-31', '2020-05-31', '2022-05-31', '2023-05-31']]
MayHomeValues = MayHomeValues.dropna(subset=['2012-05-31', '2018-05-31', '2020-05-31', '2022-05-31', '2023-05-31'])

In [4]:
#Create columns showing the growth over 1, 3, 5, and 10 years
MayHomeValues['1YearGrowth'] = ((MayHomeValues['2023-05-31'] - MayHomeValues['2022-05-31']) / MayHomeValues['2022-05-31']) * 100
MayHomeValues['3YearGrowth'] = ((MayHomeValues['2023-05-31'] - MayHomeValues['2020-05-31']) / MayHomeValues['2020-05-31']) * 100
MayHomeValues['5YearGrowth'] = ((MayHomeValues['2023-05-31'] - MayHomeValues['2018-05-31']) / MayHomeValues['2018-05-31']) * 100
MayHomeValues['10YearGrowth'] = ((MayHomeValues['2023-05-31'] - MayHomeValues['2012-05-31']) / MayHomeValues['2012-05-31']) * 100

#Columns to be Modified
columns_to_modify = ['1YearGrowth', '3YearGrowth', '5YearGrowth', '10YearGrowth']

#Convert to Percentages
MayHomeValues[columns_to_modify] = MayHomeValues[columns_to_modify].applymap(lambda x: f"{x:.1f}")

In [5]:
#Set Pandas Display Options
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')

#Creating a dataframe with the Rental Data for May
MayRentalData = rentalData[['RegionName', '2023-05-31']]

In [6]:
#Merge Rental Data to Home Value Data
merged_Rental_HomeValue = pd.merge(MayHomeValues, MayRentalData, on='RegionName')

In [7]:
#Adding Monthly Rent Columns and Calculating Annual Rent

#Removing Zip Codes where the Average Rent is Above $4500 Monthly
merged_Rental_HomeValue = merged_Rental_HomeValue[merged_Rental_HomeValue['2023-05-31_y'] <= 4500]
merged_Rental_HomeValue['Monthly Rent'] = merged_Rental_HomeValue['2023-05-31_y']
merged_Rental_HomeValue['2023-05-31_y'] = merged_Rental_HomeValue['2023-05-31_y'].multiply(12)

#Removing Zip Codes with Average Home Values Below 180K
merged_Rental_HomeValue = merged_Rental_HomeValue[merged_Rental_HomeValue['2023-05-31_x'] >= 180000]
merged_Rental_HomeValue['PriceToRentRatio'] = merged_Rental_HomeValue['2023-05-31_x'] / merged_Rental_HomeValue['2023-05-31_y']
merged_Rental_HomeValue = merged_Rental_HomeValue.rename(columns={'2023-05-31_y': 'AvgRentMay31'})

In [8]:
#Calculate Property Tax
merged_Rental_HomeValue = pd.merge(merged_Rental_HomeValue, propertyTax, on='State')
merged_Rental_HomeValue['Annual Property Tax'] = merged_Rental_HomeValue['2023-05-31_x'] * merged_Rental_HomeValue['propertyTaxRate']
merged_Rental_HomeValue['Monthly Property Tax'] = merged_Rental_HomeValue['Annual Property Tax'] / 12

In [9]:
#Convert Insurance Columns to Abbreviations
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'Washington, D.C.': 'DC',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

#Map state names to abbreviations
insuranceCostByState['State'] = insuranceCostByState['State'].map(state_abbreviations)

In [10]:
#Calculate Insurance Cost
insuranceCostByState['Cost'] = insuranceCostByState['Cost'].str.replace(',', '')  # Remove the comma from the string
insuranceCostByState['Cost'] = insuranceCostByState['Cost'].astype(float)
insuranceCostByState = insuranceCostByState.rename(columns={'Cost': 'Annual Insurance Cost'})
merged_Rental_HomeValue = pd.merge(merged_Rental_HomeValue, insuranceCostByState, on='State')
merged_Rental_HomeValue['Monthly Insurance Cost'] = merged_Rental_HomeValue['Annual Insurance Cost'] / 12

In [11]:
#Calculate Monthly Payment (20% Down Payment)

#Setting Interest Rate to 7 (Current Average)%
interest_rate7 = 7

#30 Year Mortgage
loan_term30 = 30

#Calculate the down payment for the current home values
Down_Payment_20 = (merged_Rental_HomeValue['2022-05-31'] * .20)

#Subtracting the down payment from the home values
Loan_Amount = (merged_Rental_HomeValue['2022-05-31'] - Down_Payment_20)

#Calculating the Monthly Cost
monthly_interest_rate = interest_rate7 / 12 / 100
number_of_payments = loan_term30 * 12
merged_Rental_HomeValue['Monthly Mortgage Payment'] = (Loan_Amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** (-number_of_payments))

In [12]:
#Create Monthly Cost Column
merged_Rental_HomeValue['Monthly_Payment'] = (
    merged_Rental_HomeValue['Monthly Mortgage Payment'] +
    merged_Rental_HomeValue['Monthly Property Tax'] +
    merged_Rental_HomeValue['Monthly Insurance Cost']
)

In [13]:
#Calculate Monthly Payment to Monthly Rent
merged_Rental_HomeValue['Cash Flow'] = merged_Rental_HomeValue['Monthly Rent'] - merged_Rental_HomeValue['Monthly_Payment']

In [15]:
#Test Dataframe (Optional)
pd.set_option('display.max_columns', None)
test = merged_Rental_HomeValue.drop(columns=['2012-05-31', '2018-05-31', '2020-05-31'])
sorted_df = test.sort_values('Cash Flow', ascending = False)


In [16]:
#Transfer Dataframe to CSV
sorted_df.rename(columns={'RegionName': 'Zip Code'}, inplace = True)
sorted_df.to_csv('InvestmentAnalysis.csv', index=False)


In [17]:
sorted_df

,SizeRank,Zip Code,State,City,Metro,CountyName,2022-05-31,2023-05-31_x,1YearGrowth,3YearGrowth,5YearGrowth,10YearGrowth,AvgRentMay31,Monthly Rent,PriceToRentRatio,propertyTaxRate,Annual Property Tax,Monthly Property Tax,Annual Insurance Cost,Monthly Insurance Cost,Monthly Mortgage Payment,Monthly_Payment,Cash Flow
5918,11307,3264,NH,Plymouth,"Lebanon, NH-VT",Grafton County,"326,055.937","338,619.748",3.9,51.3,68.7,88.7,"50,400.000","4,200.000",6.719,0.022,"7,381.911",615.159,736.000,61.333,"1,735.407","2,411.899","1,788.101"
3481,9054,33194,FL,NaN,"Miami-Fort Lauderdale-Pompano Beach, FL",Miami-Dade County,"527,709.620","546,503.013",3.6,34.1,41.9,133.5,"54,000.000","4,500.000",10.120,0.009,"4,863.877",405.323,"1,981.000",165.083,"2,808.692","3,379.099","1,120.901"
4156,4165,45056,OH,Oxford,"Cincinnati, OH-KY-IN",Butler County,"308,802.958","319,020.362",3.3,29.3,45.7,82.6,"39,000.000","3,250.000",8.180,0.016,"4,976.718",414.726,"1,140.000",95.000,"1,643.579","2,153.306","1,096.694"
4834,9304,19611,PA,Reading,"Reading, PA",Berks County,"184,890.579","199,630.245",8.0,38.9,64.8,99.0,"26,400.000","2,200.000",7.562,0.016,"3,154.158",262.846,760.000,63.333,984.065,"1,310.245",889.755
5683,10822,35474,AL,Moundville,"Tuscaloosa, AL",Hale County,"205,904.339","220,722.689",7.2,33.7,52.0,80.4,"24,600.000","2,050.000",8.972,0.004,904.963,75.414,"1,631.000",135.917,"1,095.909","1,307.240",742.760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,5119,94123,CA,San Francisco,"San Francisco-Oakland-Berkeley, CA",San Francisco County,"4,117,011.969","3,498,373.854",-15.0,-1.3,-1.9,72.4,"42,371.600","3,530.967",82.564,0.008,"26,587.641","2,215.637","1,225.000",102.083,"21,912.467","24,230.187","-20,699.220"
4634,22737,7723,NJ,Deal,"New York-Newark-Jersey City, NY-NJ-PA",Monmouth County,"3,115,109.376","3,444,989.013",10.6,69.0,99.5,158.7,"32,400.000","2,700.000",106.327,0.025,"85,780.226","7,148.352",775.000,64.583,"16,579.920","23,792.856","-21,092.856"
660,1962,10021,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,"3,389,580.599","4,854,642.761",43.2,64.6,319.0,2054.6,"42,581.737","3,548.478",114.008,0.017,"83,499.855","6,958.321","1,506.000",125.500,"18,040.771","25,124.593","-21,576.115"
5012,14147,2108,MA,Boston,"Boston-Cambridge-Newton, MA-NH",Suffolk County,"5,073,246.930","4,858,541.620",-4.2,7.5,8.8,108.5,"40,575.273","3,381.273",119.741,0.012,"59,760.062","4,980.005","1,199.000",99.917,"27,001.951","32,081.873","-28,700.600"
